In [2]:
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
pd.__version__

'2.2.3'

In [4]:

df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')
print(df.shape[1])

19


In [17]:
!pip install pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-02-01 00:12:18,2025-02-01 00:32:33,3.0,3.12,1.0,N,246,79,1,19.8,1.00,0.5,5.11,0.0,1.0,30.66,2.5,0.0,0.75
1,2,2025-02-01 00:40:04,2025-02-01 00:49:15,1.0,1.40,1.0,N,114,79,1,10.0,1.00,0.5,3.15,0.0,1.0,18.90,2.5,0.0,0.75
2,1,2025-02-01 00:06:09,2025-02-01 00:11:51,0.0,0.40,1.0,N,211,144,1,6.5,4.25,0.5,1.00,0.0,1.0,13.25,2.5,0.0,0.75
3,1,2025-02-01 00:15:13,2025-02-01 00:20:19,0.0,0.70,1.0,N,113,249,1,7.2,4.25,0.5,2.00,0.0,1.0,14.95,2.5,0.0,0.75
4,2,2025-02-01 00:02:52,2025-02-01 00:20:25,1.0,4.19,1.0,N,113,263,1,19.8,1.00,0.5,5.11,0.0,1.0,30.66,2.5,0.0,0.75


In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
print(df['duration'].std())

42.59435124195458


In [7]:
print(df.count)
df = df[(df.duration >= 1) & (df.duration <= 60)]
print(df.count)
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

<bound method DataFrame.count of          VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1               2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2               2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3               1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4               2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   
...           ...                  ...                   ...              ...   
3066761         2  2023-01-31 23:58:34   2023-02-01 00:12:33              NaN   
3066762         2  2023-01-31 23:31:09   2023-01-31 23:50:36              NaN   
3066763         2  2023-01-31 23:01:05   2023-01-31 23:25:36              NaN   
3066764         2  2023-01-31 23:40:00   2023-01-31 23:53:00              NaN   
3066765         2  2023-01-31 23:07:32   2023-01-31 23:21:56              Na

In [8]:
df[categorical + numerical].iloc[:10].to_dict(orient='records')

[{'PULocationID': '161', 'DOLocationID': '141', 'trip_distance': 0.97},
 {'PULocationID': '43', 'DOLocationID': '237', 'trip_distance': 1.1},
 {'PULocationID': '48', 'DOLocationID': '238', 'trip_distance': 2.51},
 {'PULocationID': '138', 'DOLocationID': '7', 'trip_distance': 1.9},
 {'PULocationID': '107', 'DOLocationID': '79', 'trip_distance': 1.43},
 {'PULocationID': '161', 'DOLocationID': '137', 'trip_distance': 1.84},
 {'PULocationID': '239', 'DOLocationID': '143', 'trip_distance': 1.66},
 {'PULocationID': '142', 'DOLocationID': '200', 'trip_distance': 11.7},
 {'PULocationID': '164', 'DOLocationID': '236', 'trip_distance': 2.95},
 {'PULocationID': '141', 'DOLocationID': '107', 'trip_distance': 3.01}]

In [9]:

# Compute standard deviation of trip distance
std_distance = df['trip_distance'].std()
print(std_distance)

251.95598137980033


In [10]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print("Feature matrix:\n", X_train)


Feature matrix:
 <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9027519 stored elements and shape (3009173, 516)>
  Coords	Values
  (0, 43)	1.0
  (0, 325)	1.0
  (0, 515)	0.97
  (1, 148)	1.0
  (1, 456)	1.0
  (1, 515)	1.1
  (2, 149)	1.0
  (2, 461)	1.0
  (2, 515)	2.51
  (3, 227)	1.0
  (3, 299)	1.0
  (3, 515)	1.9
  (4, 237)	1.0
  (4, 266)	1.0
  (4, 515)	1.43
  (5, 38)	1.0
  (5, 325)	1.0
  (5, 515)	1.84
  (6, 45)	1.0
  (6, 409)	1.0
  (6, 515)	1.66
  (7, 108)	1.0
  (7, 304)	1.0
  (7, 515)	11.7
  (8, 147)	1.0
  :	:
  (3009164, 515)	9.44
  (3009165, 54)	1.0
  (3009165, 326)	1.0
  (3009165, 515)	0.0
  (3009166, 64)	1.0
  (3009166, 482)	1.0
  (3009166, 515)	0.0
  (3009167, 46)	1.0
  (3009167, 401)	1.0
  (3009167, 515)	0.82
  (3009168, 203)	1.0
  (3009168, 266)	1.0
  (3009168, 515)	3.05
  (3009169, 233)	1.0
  (3009169, 271)	1.0
  (3009169, 515)	5.8
  (3009170, 150)	1.0
  (3009170, 273)	1.0
  (3009170, 515)	4.67
  (3009171, 237)	1.0
  (3009171, 400)	1.0
  (3009171, 515)	3.15
  (3009

In [11]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.658396898909143

In [5]:
df = pd.read_parquet('./yellow_tripdata_2023-02.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.787924973977241